In [1]:
import os

from second_brain_offline.config import settings

os.environ["OPENAI_API_KEY"] = settings.OPENAI_API_KEY

In [2]:
from second_brain_offline.application.rag import get_retriever

retriever = get_retriever()

/Users/pauliusztin/Documents/01_projects/llm-rag-bootcamp/apps/second-brain-offline/.venv-offline/lib/python3.12/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

# Retrieve and parse documents
retrieve = {
    "context": retriever
    | (lambda docs: "\n\n".join([d.page_content for d in docs])),
    "question": RunnablePassthrough(),
}
template = """Answer the question based only on the following context. If no context is provided, respond with I DON'T KNOW: \
{context}

Question: {question}
"""
# Define the chat prompt
prompt = ChatPromptTemplate.from_template(template)
# Define the model to be used for chat completion
llm = ChatOpenAI(temperature=0, model="gpt-4o-2024-11-20")
# Parse output as a string
parse_output = StrOutputParser()
# Naive RAG chain
rag_chain = retrieve | prompt | llm | parse_output

In [4]:
answer = rag_chain.invoke("How can I optimize LLMs for inference?")
print(answer)


I DON'T KNOW


In [5]:
answer = rag_chain.invoke("What is RAGAS?")
print(answer)

RAGAs (**R**etrieval-**A**ugmented **G**eneration **As**sessment) is a framework that provides the necessary tools to evaluate a Retrieval-Augmented Generation (RAG) pipeline on a component level.


In [6]:
answer = rag_chain.invoke("How does Tensorflow Recommenders work?")
print(answer)

I DON'T KNOW


In [8]:
for i, doc in enumerate(
    retriever.invoke("Evaluating RAG Applications with RAGAs")
):
    print(i, "  ", "-" * 100)
    print(doc.page_content)

0    ----------------------------------------------------------------------------------------------------
[](https://medium.com/search?source=---top_nav_layout_nav----------------------------------)

[Sign up](https://medium.com/m/signin?operation=register&redirect=https%3A%2F%2Ftowardsdatascience.com%2Fevaluating-rag-applications-with-ragas-81d67b0ee31a&source=post_page---top_nav_layout_nav-----------------------global_nav-----------)

[Sign in](https://medium.com/m/signin?operation=login&redirect=https%3A%2F%2Ftowardsdatascience.com%2Fevaluating-rag-applications-with-ragas-81d67b0ee31a&source=post_page---top_nav_layout_nav-----------------------global_nav-----------)

![](https://miro.medium.com/v2/resize:fill:64:64/1*dmbNkD5D-u45r44go_cf0g.png)

Top highlight

# Evaluating RAG Applications with RAGAs

## A framework with metrics and LLM-generated data to evaluate the performance of your Retrieval-Augmented Generation pipeline Context:
The document discusses the challenges of evaluat

In [9]:
for i, doc in enumerate(retriever.invoke("What is RAGAS?")):
    print(i, "  ", "-" * 100)
    print(doc.page_content[:100])

0    ----------------------------------------------------------------------------------------------------
Currently, determining the right evaluation metrics and collecting good validation data is an active
1    ----------------------------------------------------------------------------------------------------
[Rags](https://medium.com/tag/rags?source=post_page-----81d67b0ee31a--------------------------------
2    ----------------------------------------------------------------------------------------------------
[](https://medium.com/search?source=---top_nav_layout_nav----------------------------------)

[Sign 
